In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import joblib
os.listdir('../data')

['phase_1', 'phase_2']

# load test_170

In [2]:
test_170 = pd.read_csv('../data/phase_1/test_170.csv',index_col=0)
y_cols = test_170.iloc[:,4:].columns.tolist()
test_170['sum'] =test_170[y_cols].sum(axis=1)
test_170.head()

,T10,T50,T90,N+A,C5NP,C5IP,C5N,C6NP,C6IP,C6N,...,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,sum
0,98.6,119.2,146.1,40.133,1.142,0.616,0.217,3.745,2.577,4.028,...,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597,99.3680
1,105.4,122.2,148.2,32.175,0.122,0.078,0.029,3.955,1.563,2.407,...,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484,99.2683
2,105.8,122.7,149.2,31.428,0.098,0.064,0.020,3.920,1.447,2.296,...,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456,99.2660
3,105.4,122.4,147.6,31.390,0.122,0.085,0.021,3.900,1.437,2.274,...,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449,99.4110
4,105.4,122.6,148.2,32.418,0.072,0.049,0.015,3.873,1.369,2.206,...,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463,99.4450


# load transform matrix

In [3]:
df = pd.read_excel('../data/phase_2/FCFC Import Naphtha Composition Lab_001~100_R2.xlsx',sheet_name='Distribution')
df = df.dropna(axis=0,how='all')
df = df.dropna(axis=1,how='all')
df = df.reset_index(drop=True)
df.columns = [i for i in range(df.shape[1])]

# get idx
idx = df.iloc[:,0].values.tolist()
idx = [str(i).strip() for i in idx]
si = idx.index('O2')
ei = idx.index('TRIDECAN')
idx = idx[si:ei+1]

# get columns
cols = df.iloc[1,:].tolist()
sc = cols.index('i-P5')
ec = cols.index('A10')
cols = cols[sc:ec+1]
print(len(cols))

# get values
df = df.iloc[si:ei+1,sc:ec+1]
df = df.fillna(0)

# set index and columns
df.index = idx
df.columns = cols

# rename columns
df.columns = ['C5IP','C6IP','C6N','C7IP','C7N','C8IP','C8A','C9IP','C9N','C9A','C10IP','C10NP','C10A']

# add columns
df['C5NP'] = 0
df['C5N'] = 0
df['C6NP'] = 0
df['C6A'] = 0
df['C7NP'] = 0
df['C7A'] = 0
df['C8NP'] = 0
df['C8N'] = 0
df['C9NP'] = 0
df['C10N'] = 0

# fill values
df.loc['PENTANE','C5NP'] = 1
df.loc['CP','C5N'] = 1
df.loc['HEXANE','C6NP'] = 1
df.loc['BENZENE','C6A'] = 1
df.loc['HEPTANE','C7NP'] = 1
df.loc['TOLUENE','C7A'] = 1
df.loc['OCTANE','C8NP'] = 1
df.loc['ECH','C8N'] = 1
df.loc['NONANE','C9NP'] = 1
df.loc['TBCYHXAN','C10N'] = 1

# transpose
tr_matrix = df[y_cols].T

# check
print(tr_matrix.shape)
tr_matrix.head()

13
(23, 54)


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,...,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
C5NP,0.0,0.0,0.0,0.00,0.0,0.00,1.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C5IP,0.0,0.0,0.0,0.05,0.1,0.83,0.0,0.02,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C5N,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,1.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C6NP,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C6IP,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# check tr_matrix

In [4]:
assert tr_matrix.sum(axis=1).sum()/23 == 1.0

# build TR

In [5]:
y23 = y_cols
y54 = tr_matrix.columns.tolist()

In [6]:
class transform23to54(object):
    def __init__(self):
        self.x_cols = y23 
        self.y_cols = y54 
        self.W = tr_matrix
    def __call__(self,x):
        res = x.values@self.W.values
        return pd.DataFrame(res,columns=self.y_cols)

In [7]:
tr = transform23to54()

In [8]:
output = tr(test_170[y_cols])
output['sum'] = output.sum(axis=1)
output.head()

,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,...,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN,sum
0,0.0,0.0,0.0,0.03080,0.0616,0.51128,1.142,0.01232,0.217,2.52546,...,3.291294,0.618,0.912366,0.558254,0.172557,0.032456,0.003076,0.006290,0.0,99.3680
1,0.0,0.0,0.0,0.00390,0.0078,0.06474,0.122,0.00156,0.029,1.53174,...,3.508501,0.585,0.896433,0.452588,0.169544,0.026313,0.003023,0.005099,0.0,99.2683
2,0.0,0.0,0.0,0.00320,0.0064,0.05312,0.098,0.00128,0.020,1.41806,...,3.556251,0.543,0.867083,0.426405,0.163993,0.024790,0.002924,0.004804,0.0,99.2660
3,0.0,0.0,0.0,0.00425,0.0085,0.07055,0.122,0.00170,0.021,1.40826,...,3.525388,0.545,0.849473,0.419860,0.160662,0.024410,0.002864,0.004731,0.0,99.4110
4,0.0,0.0,0.0,0.00245,0.0049,0.04067,0.072,0.00098,0.015,1.34162,...,3.393783,0.519,0.779872,0.432951,0.147498,0.025171,0.002630,0.004878,0.0,99.4450


In [9]:
assert (output['sum'].mean()>97)&(output['sum'].mean(axis=0)<102)

# save TR

In [10]:
joblib.dump(tr,'../model/transformer(23_to_54).pkl')
print('save done!')

save done!
